[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay16.ipynb)

# --- Day 16: Ticket Translation ---

In [1]:
using System.IO;
using System.Text.RegularExpressions;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
var input = File.ReadAllText(@"input/16.txt").Split("\n\n");
var ticketFieldRules = input[0].Split("\n");
var myTicket = input[1].Split("\n").Last().Split(",").Select(int.Parse).ToArray();
var nearbyTickets = input[2].Split("\n").Skip(1).Select(
    line => line.Split(",").Select(int.Parse).ToArray()
).ToArray();

In [3]:
[Flags]
enum Fields
{
    NONE               = 0B_0000_0000_0000_0000_0000,
    DEPARTURE_LOCATION = 0B_0000_0000_0000_0000_0001,
    DEPARTURE_STATION  = 0B_0000_0000_0000_0000_0010,
    DEPARTURE_PLATFORM = 0B_0000_0000_0000_0000_0100,
    DEPARTURE_TRACK    = 0B_0000_0000_0000_0000_1000,
    DEPARTURE_DATE     = 0B_0000_0000_0000_0001_0000,
    DEPARTURE_TIME     = 0B_0000_0000_0000_0010_0000,
    ARRIVAL_LOCATION   = 0B_0000_0000_0000_0100_0000,
    ARRIVAL_STATION    = 0B_0000_0000_0000_1000_0000,
    ARRIVAL_PLATFORM   = 0B_0000_0000_0001_0000_0000,
    ARRIVAL_TRACK      = 0B_0000_0000_0010_0000_0000,
    CLASS              = 0B_0000_0000_0100_0000_0000,
    DURATION           = 0B_0000_0000_1000_0000_0000,
    PRICE              = 0B_0000_0001_0000_0000_0000,
    ROUTE              = 0B_0000_0010_0000_0000_0000,
    ROW                = 0B_0000_0100_0000_0000_0000,
    SEAT               = 0B_0000_1000_0000_0000_0000,
    TRAIN              = 0B_0001_0000_0000_0000_0000,
    TYPE               = 0B_0010_0000_0000_0000_0000,
    WAGON              = 0B_0100_0000_0000_0000_0000,
    ZONE               = 0B_1000_0000_0000_0000_0000,
}

In [4]:
record Range(int Lower, int Upper);

In [5]:
record Rule
{
    public Fields FieldName { get; init; }
    public List<Range> Ranges { get; init; }
    
    public Rule(string rule)
    {
        var split = rule.Split(": ");
        var pattern = @"(\d+)-(\d+)";
        
        FieldName = Enum.Parse<Fields>(split.First().Replace(" ", "_").ToUpper());
        Ranges = Regex.Matches(split.Last(), pattern).Select(
            match => new Range(
                Convert.ToInt32(match.Groups[1].Value),
                Convert.ToInt32(match.Groups[2].Value)
            )
        ).ToList();
    }
}

In [6]:
var rules = ticketFieldRules.Select(rule => new Rule(rule));
rules

index,FieldName,Ranges
0,DEPARTURE_LOCATION,"[ Range { Lower = 49, Upper = 239 }, Range { Lower = 247, Upper = 960 } ]"
1,DEPARTURE_STATION,"[ Range { Lower = 43, Upper = 135 }, Range { Lower = 155, Upper = 963 } ]"
2,DEPARTURE_PLATFORM,"[ Range { Lower = 27, Upper = 426 }, Range { Lower = 449, Upper = 955 } ]"
3,DEPARTURE_TRACK,"[ Range { Lower = 43, Upper = 655 }, Range { Lower = 680, Upper = 949 } ]"
4,DEPARTURE_DATE,"[ Range { Lower = 49, Upper = 159 }, Range { Lower = 175, Upper = 970 } ]"
5,DEPARTURE_TIME,"[ Range { Lower = 44, Upper = 257 }, Range { Lower = 280, Upper = 970 } ]"
6,ARRIVAL_LOCATION,"[ Range { Lower = 26, Upper = 825 }, Range { Lower = 848, Upper = 950 } ]"
7,ARRIVAL_STATION,"[ Range { Lower = 25, Upper = 549 }, Range { Lower = 557, Upper = 956 } ]"
8,ARRIVAL_PLATFORM,"[ Range { Lower = 50, Upper = 460 }, Range { Lower = 486, Upper = 964 } ]"
9,ARRIVAL_TRACK,"[ Range { Lower = 50, Upper = 368 }, Range { Lower = 385, Upper = 950 } ]"


In [7]:
bool IsValidForAnyField(int input, IEnumerable<Rule> rules)
{
    foreach (var rule in rules)
    {
        foreach (var range in rule.Ranges)
        {
            if (range.Lower <= input && input <= range.Upper)
            {
                return true;
            }
        }
    }
    
    return false;
}

In [8]:
nearbyTickets.SelectMany(ticket => ticket).Where(input => !IsValidForAnyField(input, rules)).Sum()

22977

# --- Part Two ---

In [9]:
bool IsTicketValid(int[] ticket, IEnumerable<Rule> rules) =>
    !ticket.Where(input => !IsValidForAnyField(input, rules)).Any();

In [10]:
bool IsSingleField(Fields field) =>
    field != 0 && (field & (field - 1)) == 0;

In [11]:
var validTickets = nearbyTickets.Where(ticket => IsTicketValid(ticket, rules));
validTickets.Count()

190

In [12]:
Fields[] FindFieldOrder(IEnumerable<int[]> validTickets, IEnumerable<Rule> rules)
{
    var fieldOrder = Enumerable.Range(0, rules.Count())
        .Select(index => (Fields)(0B_1111_1111_1111_1111_1111))
        .ToArray();
    
    foreach (var ticket in validTickets)
    {
        for (var i = 0; i < ticket.Length; i++)
        {
            foreach (var rule in rules)
            {
                var valid = false;
                foreach (var range in rule.Ranges)
                {
                    if (range.Lower <= ticket[i] && ticket[i] <= range.Upper)
                    {
                        valid = true;
                        break;
                    }
                }
                
                if (!valid)
                {
                    fieldOrder[i] = fieldOrder[i] ^ rule.FieldName;
                }
            }
        }
    }
    
    var fieldsWithSingleFlag = fieldOrder.Where(field => IsSingleField(field));
    while (fieldsWithSingleFlag.Count() != fieldOrder.Count())
    {
        foreach (var singleFlag in fieldsWithSingleFlag)
        {
            for (var i = 0; i < fieldOrder.Count(); i++)
            {
                var field = fieldOrder[i];
                if (!IsSingleField(field) && field.HasFlag(singleFlag))
                {
                    fieldOrder[i] = field ^ singleFlag;
                }
            }
        }
    }
    
    return fieldOrder;
}

In [13]:
var fieldOrder = FindFieldOrder(validTickets, rules);

for (var i = 0; i < fieldOrder.Count(); i++)
{
    Console.WriteLine($"Index: {i}, Field: {fieldOrder[i]}");
}

Index: 0, Field: ROW
Index: 1, Field: DEPARTURE_TRACK
Index: 2, Field: DURATION
Index: 3, Field: ARRIVAL_STATION
Index: 4, Field: DEPARTURE_DATE
Index: 5, Field: CLASS
Index: 6, Field: TYPE
Index: 7, Field: ARRIVAL_LOCATION
Index: 8, Field: PRICE
Index: 9, Field: TRAIN
Index: 10, Field: SEAT
Index: 11, Field: ARRIVAL_PLATFORM
Index: 12, Field: ZONE
Index: 13, Field: DEPARTURE_LOCATION
Index: 14, Field: DEPARTURE_STATION
Index: 15, Field: DEPARTURE_PLATFORM
Index: 16, Field: ROUTE
Index: 17, Field: DEPARTURE_TIME
Index: 18, Field: ARRIVAL_TRACK
Index: 19, Field: WAGON


In [14]:
fieldOrder
    .Zip(myTicket, (f, v) => (Field: Enum.GetName<Fields>(f), Value: v))
    .Where(t => t.Field.StartsWith("DEPARTURE"))
    .Select(t => (long)t.Value)
    .Aggregate((a, b) => a*b)

998358379943